In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader

ModuleNotFoundError: No module named 'torch'

In [6]:
data_path = "../NER_Products_model/WordNER.csv"

data = pd.read_csv(data_path, encoding= 'unicode_escape')

# data.fillna(method = 'ffill', inplace = True)
data.head()

,Sentence #,Word,Tag
0,Sentence: 1,Milly,B-BRAND
1,Sentence: 1,24,O
2,Sentence: 1,SLICES,O
3,Sentence: 1,CHEESE,B-PRODUCT
4,Sentence: 1,SLICES,I-PRODUCT


In [7]:
ready_dist_path = "../NER_Products_model/SentenceNER.csv"
ready_data = pd.read_csv(ready_dist_path)
ready_data.head()

,Sentence #,Sentence,Tag
0,Sentence: 1,Milly 24 SLICES CHEESE SLICES Net Weight : 480...,"['B-BRAND', 'O', 'O', 'B-PRODUCT', 'I-PRODUCT'..."
1,Sentence: 2,SPICED BUTTERMILK 9.00 Ne l 26 ) 2pcs Amu . ( ...,"['B-PRODUCT', 'I-PRODUCT', 'O', 'O', 'O', 'O',..."
2,Sentence: 3,Partly Skimmed Fruit Yogurt epigamia eplgmia 1...,"['O', 'O', 'O', 'B-PRODUCT', 'B-BRAND', 'O', '..."
3,Sentence: 4,modeRn moder DeRn Omilk plus bread W 's Co pac...,"['O', 'O', 'O', 'O', 'O', 'B-PRODUCT', 'O', 'O..."
4,Sentence: 5,BAREVR 2 iDY PANEER SOFT & CREAMYC FROM NATURA...,"['O', 'O', 'B-BRAND', 'B-PRODUCT', 'O', 'O', '..."


In [8]:
def join_a_sentence(sentence_number):
    
    sentence_number = str(sentence_number)
    the_sentence_words_list = list(data[data['Sentence #'] == 'Sentence: {}'.format(sentence_number)]['Word'])
    
    return ' '.join(the_sentence_words_list)

In [9]:
join_a_sentence(sentence_number = 1)

'Milly 24 SLICES CHEESE SLICES Net Weight : 480 g 0.53 KB / S " Suggested Serving ony 26'

In [10]:
tags = data.Tag.unique()
tags

array(['B-BRAND', 'O', 'B-PRODUCT', 'I-PRODUCT', 'I-BRAND', nan],
      dtype=object)

In [11]:
def num_words_tags (tags, data):
    
    tags_count = {}
    
    for tag in tags:
        len_tag = len(data[data['Tag'] == tag])
        tags_count[tag] = len_tag
    
    return tags_count

tags_count = num_words_tags(tags, data)
tags_count

{'B-BRAND': 101,
 'O': 3393,
 'B-PRODUCT': 110,
 'I-PRODUCT': 87,
 'I-BRAND': 28,
 nan: 0}

In [12]:
X = list(ready_data['Sentence'])
Y = list(ready_data['Tag'])

In [13]:
print(ready_data['Tag'])

0      ['B-BRAND', 'O', 'O', 'B-PRODUCT', 'I-PRODUCT'...
1      ['B-PRODUCT', 'I-PRODUCT', 'O', 'O', 'O', 'O',...
2      ['O', 'O', 'O', 'B-PRODUCT', 'B-BRAND', 'O', '...
3      ['O', 'O', 'O', 'O', 'O', 'B-PRODUCT', 'O', 'O...
4      ['O', 'O', 'B-BRAND', 'B-PRODUCT', 'O', 'O', '...
                             ...                        
110    ['B-BRAND', 'O', 'B-PRODUCT', 'I-PRODUCT', 'O'...
111    ['B-PRODUCT', 'I-PRODUCT', 'O', 'O', 'O', 'O',...
112    ['O', 'O', 'O', 'B-PRODUCT', 'B-BRAND', 'O', '...
113    ['B-BRAND', 'O', 'O', 'O', 'O', 'O', 'O', 'O',...
114    ['O', 'O', 'B-BRAND', 'B-PRODUCT', 'O', 'O', '...
Name: Tag, Length: 115, dtype: object


In [14]:
Y_ready = []
for sen_tags in Y:
    # Handle missing values (replace with your preferred approach)
    if sen_tags == 'nan':
        Y_ready.append([])
        continue

    # Remove leading/trailing characters (if needed)
    clean_str = sen_tags[1:-1]  # Assuming quotes enclose the list

    # Check if the string is empty after cleaning (potential missing value)
    if not clean_str:
        Y_ready.append([])  # Append an empty list for empty strings
        continue

    # Split by delimiter (comma in this case)
    tag_list = clean_str.split(',')

    # Remove quotes or leading/trailing spaces from individual tags (if needed)
    tag_list = [tag.strip()[1:-1] if len(tag) > 2 else tag.strip() for tag in tag_list]

    Y_ready.append(tag_list)

print(Y_ready[:3])

[['B-BRAND', 'O', 'O', 'B-PRODUCT', 'I-PRODUCT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-PRODUCT', 'I-PRODUCT', 'O', 'O', 'O', 'O', 'O', 'O', 'B-BRAND', 'I-BRAND', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'B-PRODUCT', 'B-BRAND', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


In [21]:
label2id = {k: v for v, k in enumerate(data.Tag.unique())}
id2label = {v: k for v, k in enumerate(data.Tag.unique())}
label2id

{'B-BRAND': 0, 'O': 1, 'B-PRODUCT': 2, 'I-PRODUCT': 3, 'I-BRAND': 4, nan: 5}

In [22]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

In [2]:
from transformers import DistilBertTokenizerFast

In [23]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# pad_token = tokenizer.pad_token 

# def prepare_data(sentences, tags, max_len=50):
#   tokenized_sentences = []
#   attention_masks = []
#   padded_labels = []

#   # Loop through sentences and tags (using unpacking from zip)
#   for sentence, tag_list in zip(sentences, tags):
#     # Access the current tag for each sentence
#     tag = tag_list[0]  # Assuming tags are single strings (adjust if they are lists)

#     # Tokenize the sentence
#     encoded_sentence = tokenizer(sentence, add_special_tokens=True, padding='max_length', truncation=True)

#     # Extract token IDs, attention mask, and pad tags
#     tokenized_sentences.append(encoded_sentence['input_ids'])
#     attention_masks.append(encoded_sentence['attention_mask'])
#     padded_labels.append([tag] + [pad_token] * (max_len - len(tag_list)))  # Pad shorter tag sequences

#   # Convert lists to NumPy arrays
#   tokenized_sentences = np.array(tokenized_sentences, dtype=np.int32)
#   attention_masks = np.array(attention_masks, dtype=np.int32)
#   labels = np.array(padded_labels, dtype=np.int32)  # Assuming numerical tag IDs

#   return tokenized_sentences, attention_masks, labels

# tokenized_sentences, attention_masks, labels = prepare_data(X, Y_ready)

# # Now you have your data prepared for training your NER model with DistilBERT
# print(f"Tokenized sentences: {tokenized_sentences.shape}")
# print(f"Attention masks: {attention_masks.shape}")
# print(f"Labels: {labels.shape}")

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels